# A Model of the Chorus Hall

## Imports

In [ ]:
import os
from itertools import repeat
from multiprocessing import Pool
import numpy as np
import pandas as pd
import pickle
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from apsrm import Box
from apsrm.config import POOL_NCORES, DEFAULT_STRAIN
from apsrm.ext.multiprocessing import ProcessSeeder
from apsrm.ext.simulation import (
    create_pathogen,
    run_simulation,
    generate_means_tables)
from utils import (
    BOX_TYPE,
    create_workplace,
    create_emissions_calculator)

pathogen_name = "wt"
R = 10000
n_singers = 61

OUTPUT_BASE_DIR = "../../../jrsi-article/figs/outputs/choral"
OUTPUT_BASE_DIR = OUTPUT_BASE_DIR if os.path.exists(OUTPUT_BASE_DIR) else "."
def opath(p): return os.path.join(OUTPUT_BASE_DIR, p)

output_pickle = os.path.join(OUTPUT_BASE_DIR, 'all_results.pkl')
run_analyses = not os.path.exists(output_pickle)

process_seeder = ProcessSeeder()

def infectivity(*args, **kwargs):
    return 1.0

def run_job(runner, R, intervention_name):
    if run_analyses:
        process_seeder.reset()
        with Pool(POOL_NCORES, initializer=process_seeder) as pool:
            work = pool.imap_unordered(runner, range(R))
            return pd.DataFrame(tqdm(work, total=R)).assign(intervention=intervention_name)

def standard_runner(*args):
    external_acp = np.random.uniform(0.3, 1.0, size=1)
    dieoff_rate = np.random.uniform(0.0, 0.63, size=1) + np.random.uniform(0.3, 1.5, size=1)
    breathing_rate = np.random.uniform(0.65, 1.38, size=1)
    pathogen = create_pathogen(pathogen_name, dieoff_rate=dieoff_rate, infectivity=infectivity)
    emissions_calculator = create_emissions_calculator(breathing_rate)
    workplace = create_workplace(n_singers=n_singers, single_box=True)
    default_external_ventilation = [0. if b.use == BOX_TYPE.HVAC else (external_acp * b.volume) for b in workplace.boxes]
    workplace.set_ventilation_properties(
        force_standard_hvac_system = True, 
        external_ventilation = default_external_ventilation)
    return run_simulation(workplace, pathogen, emissions_calculator, stop_at_end_of_period=0)

## Simulations

In [ ]:
if run_analyses:
    results_bau = run_job(standard_runner, R, 'BAU')
    with open(output_pickle, 'wb') as pkl:
        pickle.dump((results_bau, R), pkl)

else:
    with open(output_pickle, 'rb') as pkl:
        results_bau, R = pickle.load(pkl)

## Plots and Tables

In [ ]:
import matplotlib.font_manager as font_manager

# Add every font at the specified location
font_dir = ['../fnt']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Palatino'

In [ ]:
def plot_histogram(d,column):
    d.hist(
        column=column,
        align='mid',                                                                                                                                                                                
        color='gray', 
        density=True,
        bins=[b - .5 for b in range(20, 62)])

plot_histogram(results_bau, "number_infected")
plt.title('Number Infected')
plt.savefig(opath('number_infected_hist.pdf'))#, bbox_inches='tight')

In [ ]:
np.mean(results_bau["number_infected"])

In [ ]:
np.std(results_bau["number_infected"], ddof=1)